# Searching A Single Document By Rijkswaterstaat (RWS)

A test on how to load a PDF document and search it using the Perry AI Powered Document Analysis tool.

## Constructing The File Nodes

In [7]:
# Check if doc_data/RWS exists, give warning if not
import pathlib

def get_data_path():
    """Returns the path to the RWS data directory."""
    return pathlib.Path.cwd() / "doc_data" / "RWS"

rws_path = get_data_path()
if not rws_path.exists():
    print("Warning: RWS data not found. Please create the directory and fill it with document data first.")

if not any(rws_path.iterdir()):
    print("Warning: RWS data not found. Please fill the directory with document data first.")

In [8]:
import os
import openai
from dotenv import load_dotenv
from llama_index import VectorStoreIndex, StorageContext, ServiceContext, load_index_from_storage
from llama_index.readers.base import BaseReader
from llama_index.schema import Document

def load_openai_api_key():
    """Load OpenAI API key from environment variable."""
    load_dotenv()
    openai.api_key = os.environ["OPENAI_API_KEY"]

def get_rws_file_paths(data_dir: pathlib.Path) -> list[pathlib.Path]:
    """Get file paths of RWS documents."""
    return [f for f in data_dir.iterdir() if f.is_file()]

def load_rws_documents(data_dir: pathlib.Path, loader: BaseReader) -> dict[str, list[Document]]:
    """Load RWS documents."""
    doc_set = {}
    for doc_path in get_rws_file_paths(data_dir):
        docs = loader.load_data(file=doc_path, split_documents=False)
        doc_set[doc_path.stem] = docs
    return doc_set
    
def create_vector_indexes(doc_set: dict[str, list[Document]]) -> dict[str, list[VectorStoreIndex]]:
    """Create and store vector indexes."""
    service_context = ServiceContext.from_defaults(chunk_size=512)
    vector_indexes = {}
    for doc_name, docs in doc_set.items():
        storage_context = StorageContext.from_defaults()
        vector_index = VectorStoreIndex.from_documents(
            docs,
            service_context=service_context,
            storage_context=storage_context
        )
        vector_indexes[doc_name] = vector_index
    return vector_indexes

def save_vector_indexes(vector_indexes: dict[str, list[VectorStoreIndex]]) -> None:
    """Save vector indexes."""
    for doc_name, vector_index in vector_indexes.items():
        vector_index.storage_context.persist(persist_dir=f'./storage/rws/{doc_name}')

def load_vector_indexes() -> dict[str, list[VectorStoreIndex]]:
    """Load vector indexes."""
    vector_indexes = {}
    for doc_name in get_rws_file_paths(rws_path):
        storage_context = StorageContext.from_defaults(persist_dir=f'./storage/rws/{doc_name.stem}')
        vector_index = load_index_from_storage(storage_context=storage_context)
        vector_indexes[doc_name.stem] = vector_index
    return vector_indexes


In [5]:
# Load in docs
from llama_index import download_loader
load_openai_api_key()
UnstructuredReader = download_loader("UnstructuredReader", refresh_cache=True)
loader = UnstructuredReader()
docs = load_rws_documents(rws_path, loader)

[nltk_data] Downloading package punkt to /home/bluegnome/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/bluegnome/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [9]:
vector_indexes_new = create_vector_indexes(docs)
save_vector_indexes(vector_indexes_new)

In [10]:
vector_indexes = load_vector_indexes()

In [11]:
vector_indexes_new.keys()

dict_keys(['Aanbestedingsleidraad_BOC_ZD'])

In [12]:
vector_indexes.keys()

dict_keys(['Aanbestedingsleidraad_BOC_ZD'])

# Create Langchain Tooling

Create langchain tooling from the loaded indexes.

In [34]:
from langchain.chains.conversation.memory import ConversationBufferMemory

from llama_index.langchain_helpers.agents import LlamaToolkit, create_llama_chat_agent, IndexToolConfig
from llama_index.bridge.langchain import AgentExecutor

from langchain import OpenAI


def create_vector_index_tool_configs(vector_indexes: dict[str, list[VectorStoreIndex]]) -> dict[str, IndexToolConfig]:
    """Create vector index tool configs."""
    index_tool_configs = {}
    for doc_name, vector_index in vector_indexes.items():
        query_engine = vector_index.as_query_engine(
            similarity_top_k=3,
        )
        index_tool_configs[doc_name] = IndexToolConfig(
            query_engine=query_engine,
            name=f"Vector Index {doc_name}",
            description=f"Aanbestedingsleidraad voor rijkswaterstaat opdrachten",
            tool_kwargs={"return_direct": True}
        )
    return index_tool_configs

def create_langchain_toolkit(index_tool_configs: dict[str, IndexToolConfig]) -> LlamaToolkit:
    """Create LangChain toolkit."""
    return LlamaToolkit(index_configs= list(index_tool_configs.values()))

def create_langchain_chat_agent(toolkit: LlamaToolkit) -> AgentExecutor:
    """Create LangChain chat agent."""
    memory = ConversationBufferMemory(memory_key="chat_history")
    llm=OpenAI(temperature=0)
    return create_llama_chat_agent(
        toolkit,
        llm,
        memory=memory,
        verbose=True
    )


In [35]:
toolkit = create_langchain_toolkit(create_vector_index_tool_configs(vector_indexes))
agent = create_langchain_chat_agent(toolkit)

# Ask Questions For Testing

In [20]:
agent.run(input="Hi GPT, I'm Arthur, king of the britons.")



> Entering new  chain...

Thought: Do I need to use a tool? No
AI: Hi Arthur, it's nice to meet you! What can I do for you today?

> Finished chain.


"Hi Arthur, it's nice to meet you! What can I do for you today?"

In [22]:
answer = agent.run(input="Give me a bullet point list of the three most important aspects that rijkswaterstaat considers when awarding a contract in dutch.")



> Entering new  chain...

AI: 
1. Uitsluitingsgronden: Rijkswaterstaat neemt alle uitsluitingsgronden in acht die een inschrijver kunnen diskwalificeren voor de opdracht.

2. Geschiktheidseisen: Rijkswaterstaat beoordeelt de geschiktheidseisen van de inschrijver om te bepalen of deze geschikt is om de opdracht uit te voeren.

3. Bewijsmiddelen: Rijkswaterstaat beoordeelt de bewijsmiddelen die de inschrijver heeft aangeleverd om te bewijzen dat deze in staat is om de opdracht uit te voeren.

> Finished chain.


In [24]:
# Display markdown answer
from IPython.display import Markdown
Markdown(answer)

1. Uitsluitingsgronden: Rijkswaterstaat neemt alle uitsluitingsgronden in acht die een inschrijver kunnen diskwalificeren voor de opdracht.

2. Geschiktheidseisen: Rijkswaterstaat beoordeelt de geschiktheidseisen van de inschrijver om te bepalen of deze geschikt is om de opdracht uit te voeren.

3. Bewijsmiddelen: Rijkswaterstaat beoordeelt de bewijsmiddelen die de inschrijver heeft aangeleverd om te bewijzen dat deze in staat is om de opdracht uit te voeren.

In [31]:
answer = agent.run(input="Vat voor mij de aanbestedingsleidraad voor rijkswaterstaat opdrachten samen in een paar punten.")



> Entering new  chain...

AI: De aanbestedingsleidraad voor rijkswaterstaat opdrachten bevat de volgende punten:

1. Uitsluitingsgronden: Rijkswaterstaat neemt alle uitsluitingsgronden in acht die een inschrijver kunnen diskwalificeren voor de opdracht.

2. Geschiktheidseisen: Rijkswaterstaat beoordeelt de geschiktheidseisen van de inschrijver om te bepalen of deze geschikt is om de opdracht uit te voeren.

3. Bewijsmiddelen: Rijkswaterstaat beoordeelt de bewijsmiddelen die de inschrijver heeft aangeleverd om te bewijzen dat deze in staat is om de opdracht uit te voeren.

4. Planning: Rijkswaterstaat hanteert een standaardtermijn van maximaal zes maanden v

> Finished chain.


In [26]:
Markdown(answer)

Volgens Rijkswaterstaat moet een aanmelding voor een contract door de volgende fases gaan:

1. Aanmelding: Hierbij dient de inschrijver zich aan te melden voor de opdracht.

2. Beoordeling: Rijkswaterstaat beoordeelt de aanmelding op basis van de uitsluitingsgronden, geschiktheidseisen en bewijsmiddelen.

3. Selectie: Rijkswaterstaat selecteert de inschrijver die het beste aan de eisen voldoet.

4. Gunning: Rijkswaterstaat gunt de opdracht aan de geselecteerde inschrijver.

In [43]:
answer = agent.run(input="Welke documenten moeten bij aanmelding verstrek worden volgens het aanbestedingsleidraad document voor rijkswaterstaat?")



> Entering new  chain...

AI: Volgens het aanbestedingsleidraad document voor rijkswaterstaat moeten bij aanmelding de volgende documenten verstrekt worden: een verklaring van deelname, een verklaring van geschiktheid, een verklaring van financiële draagkracht, een verklaring van technische draagkracht, een verklaring van kwaliteit en een verklaring van milieudraagkracht.

> Finished chain.
